In [1]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable as Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.model_zoo as model_zoo
from torch.optim import lr_scheduler
from collections import OrderedDict
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os

plt.ion()

In [2]:
train = pd.read_csv("train_df.csv")

In [3]:
train.shape

(42000, 785)

In [4]:
test = pd.read_csv("test_df.csv")

In [5]:
test.head()

,Unnamed: 0,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,57808,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4960,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,35755,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15543,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,48968,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test.shape

(18000, 785)

In [7]:
X = train.iloc[:,1:]

In [8]:
X = X / 255

In [10]:
Y =train.iloc[:,0]

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_val,Y_train,Y_val = train_test_split(X, Y, train_size=0.85,random_state=10, stratify = train.label)

In [12]:
pd.DataFrame(pd.concat([X_train,Y_train],axis=1)).to_csv("train.csv", index =False)

In [13]:
pd.DataFrame(pd.concat([X_val,Y_val],axis=1)).to_csv("val.csv", index = False)

In [14]:
from torch.utils.data import DataLoader

In [15]:
class BengaliDataset(Dataset):
    def __init__(self, csv, transform=None):
#         super(BengaliDataset, self).__init__(transform=transform)
        self.csv = pd.read_csv(csv)
        self.image_ids = self.csv.index
        self.transform = transform
        
    def __len__(self):
        return len(self.csv)
    
    def __getitem__(self, index):
        image = self.csv.iloc[index,:-1]   
        label = self.csv.iloc[index,-1]         
        return (torch.tensor(image, dtype=torch.float), torch.tensor(label, dtype=torch.long))

In [16]:
trainset = BengaliDataset("train.csv")
valset = BengaliDataset("val.csv")

In [17]:
train_loader = DataLoader(trainset,batch_size = 128, shuffle= True)
test_loader = DataLoader(valset,batch_size = 128, shuffle= True) # validation

### 모델 설명
MINIST로 학습된 pretrained 모델을 불러와서 fine tunning을 해보자.   
MLP로 구축된 모델로, MINIST에서 약 98%의 정확도를 보여줌.

http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth' 에 저장되어 있는 weight을 가져와서 layer에 initailization.

In [18]:
class YEJI(nn.Module):
    def __init__(self, input_dims, n_hiddens, n_class):
        super(YEJI, self).__init__()
        self.input_dims = input_dims
        current_dims = input_dims
        layers = OrderedDict()

        if isinstance(n_hiddens, int):
            n_hiddens = [n_hiddens]
        else:
            n_hiddens = list(n_hiddens)
        for i, n_hidden in enumerate(n_hiddens):
            layers['fc{}'.format(i+1)] = nn.Linear(current_dims, n_hidden)
            layers['relu{}'.format(i+1)] = nn.ReLU()
            layers['drop{}'.format(i+1)] = nn.Dropout(0.2)
            current_dims = n_hidden
        layers['out'] = nn.Linear(current_dims, n_class)

        self.model= nn.Sequential(layers)
        print(self.model)

    def forward(self, input):
        return self.model.forward(input)

def TOBIGS(input_dims=784, n_hiddens=[256, 256], n_class=10):
    model = YEJI(input_dims, n_hiddens, n_class)
    model_urls = {'mnist': 'http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth'}
    m = model_zoo.load_url(model_urls['mnist'])
    state_dict = m.state_dict() if isinstance(m, nn.Module) else m
    assert isinstance(state_dict, (dict, OrderedDict)), type(state_dict)

    model.load_state_dict(state_dict)
    return model

In [19]:
model = TOBIGS(input_dims=784, n_hiddens=[256, 256], n_class=10)
model = torch.nn.DataParallel(model)
model.cuda()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Sequential(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0.2, inplace=False)
  (out): Linear(in_features=256, out_features=10, bias=True)
)


- 초기화된 parameters 확인

In [20]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[ 0.0008,  0.0228, -0.0091,  ...,  0.0253,  0.0081,  0.0273],
        [-0.0286, -0.0235,  0.0097,  ..., -0.0235, -0.0198,  0.0118],
        [-0.0265,  0.0142, -0.0108,  ..., -0.0061,  0.0135, -0.0146],
        ...,
        [ 0.0200, -0.0206,  0.0216,  ...,  0.0163, -0.0218, -0.0243],
        [-0.0293, -0.0149, -0.0179,  ...,  0.0208,  0.0032, -0.0182],
        [ 0.0217, -0.0209, -0.0171,  ...,  0.0146, -0.0300, -0.0282]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-2.4047e-02, -2.2005e-02, -2.2303e-02,  1.4438e-02, -3.4430e-02,
         2.8466e-02, -3.1873e-03, -3.6541e-02, -3.6213e-02,  9.2949e-03,
        -2.6346e-03, -1.6759e-03, -2.0798e-02,  1.2568e-02, -2.7282e-02,
        -2.4870e-02,  1.2711e-02,  9.0569e-03,  3.1330e-02,  7.8664e-03,
        -9.7114e-03, -5.3195e-03,  6.5779e-03,  4.8497e-03, -1.6000e-02,
         3.4190e-02, -4.3266e-02, -5.3593e-03, -2.5096e-02, -3.0591e-02,
         8.3767e-03,  2.1216e-02, -2.967

In [21]:
def expand_user(path):
    return os.path.abspath(os.path.expanduser(path))

def model_snapshot(model, new_file, old_file=None, verbose = True):
    if isinstance(model, torch.nn.DataParallel):
        model = model.module
    if old_file and os.path.exists(expand_user(old_file)):
        print("Removing old model {}".format(expand_user(old_file)))
        os.remove(expand_user(old_file))
    if verbose:
        print("Saving model to {}".format(expand_user(new_file)))

    state_dict = OrderedDict()
    for k, v in model.state_dict().items():
        if v.is_cuda:
            v = v.cpu()
        state_dict[k] = v
    torch.save(state_dict, expand_user(new_file))

In [24]:
best_acc = 0.0
old_file = None

In [23]:
for epoch in range(10):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            indx_target = target.clone()
            data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)

            optimizer.zero_grad()
            output = model(data)
#             output = nn.Softmax(output)
#             loss = F.mse_loss(output, target)
            loss=F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 10 ==0 and batch_idx > 0:
                pred = output.data.max(1)[1]  # get the index of the max log-probability
                correct = pred.cpu().eq(indx_target).sum()
                acc = correct * 1.0 / len(data)
                print('Train Epoch: {} [{}/{}] Loss: {:.6f} Acc: {:.4f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    loss, acc))

        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            indx_target = target.clone()
            data, target = data.cuda(), target.cuda()
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            test_loss += F.cross_entropy(output, target)
            pred = output.data.max(1)[1]
            correct += pred.cpu().eq(indx_target).sum()

        test_loss = test_loss / len(test_loader) 
        acc = 100. * correct / float(len(test_loader.dataset))
        print('\tTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            test_loss, correct, len(test_loader.dataset), acc))
        
        if acc > best_acc:
            new_file = os.path.join('./', 'best-{}.pth'.format(epoch))
            model_snapshot(model, new_file, old_file=old_file)
            best_acc = acc
            old_file = new_file

C:\Users\fdal\Miniconda3\envs\torch\lib\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


Train Epoch: 0 [1280/35700] Loss: 2.411509 Acc: 0.2500
Train Epoch: 0 [2560/35700] Loss: 2.049485 Acc: 0.3281
Train Epoch: 0 [3840/35700] Loss: 1.733531 Acc: 0.3828
Train Epoch: 0 [5120/35700] Loss: 1.725729 Acc: 0.4453
Train Epoch: 0 [6400/35700] Loss: 1.459448 Acc: 0.5000
Train Epoch: 0 [7680/35700] Loss: 1.327650 Acc: 0.5312
Train Epoch: 0 [8960/35700] Loss: 1.050043 Acc: 0.7500
Train Epoch: 0 [10240/35700] Loss: 0.945391 Acc: 0.6953
Train Epoch: 0 [11520/35700] Loss: 0.903133 Acc: 0.6875
Train Epoch: 0 [12800/35700] Loss: 0.920729 Acc: 0.7266
Train Epoch: 0 [14080/35700] Loss: 0.612463 Acc: 0.8750
Train Epoch: 0 [15360/35700] Loss: 0.730327 Acc: 0.8047
Train Epoch: 0 [16640/35700] Loss: 0.532178 Acc: 0.8828
Train Epoch: 0 [17920/35700] Loss: 0.490356 Acc: 0.8828
Train Epoch: 0 [19200/35700] Loss: 0.559360 Acc: 0.8359
Train Epoch: 0 [20480/35700] Loss: 0.412116 Acc: 0.8672
Train Epoch: 0 [21760/35700] Loss: 0.362199 Acc: 0.9062
Train Epoch: 0 [23040/35700] Loss: 0.464376 Acc: 0.8672

C:\Users\fdal\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


	Test set: Average loss: 0.2049, Accuracy: 5982/6300 (95%)
Saving model to C:\Users\fdal\Desktop\bigs\TOBIGS\best-0.pth
Train Epoch: 1 [1280/35700] Loss: 0.279817 Acc: 0.9375
Train Epoch: 1 [2560/35700] Loss: 0.150849 Acc: 0.9609
Train Epoch: 1 [3840/35700] Loss: 0.354600 Acc: 0.9062
Train Epoch: 1 [5120/35700] Loss: 0.259003 Acc: 0.9297
Train Epoch: 1 [6400/35700] Loss: 0.284537 Acc: 0.9453
Train Epoch: 1 [7680/35700] Loss: 0.151071 Acc: 0.9688
Train Epoch: 1 [8960/35700] Loss: 0.200743 Acc: 0.9609
Train Epoch: 1 [10240/35700] Loss: 0.154068 Acc: 0.9609
Train Epoch: 1 [11520/35700] Loss: 0.291180 Acc: 0.9219
Train Epoch: 1 [12800/35700] Loss: 0.214344 Acc: 0.9609
Train Epoch: 1 [14080/35700] Loss: 0.279012 Acc: 0.9062
Train Epoch: 1 [15360/35700] Loss: 0.146066 Acc: 0.9609
Train Epoch: 1 [16640/35700] Loss: 0.146616 Acc: 0.9688
Train Epoch: 1 [17920/35700] Loss: 0.211097 Acc: 0.9375
Train Epoch: 1 [19200/35700] Loss: 0.163379 Acc: 0.9609
Train Epoch: 1 [20480/35700] Loss: 0.222867 Acc

Train Epoch: 5 [32000/35700] Loss: 0.096192 Acc: 0.9609
Train Epoch: 5 [33280/35700] Loss: 0.149671 Acc: 0.9688
Train Epoch: 5 [34560/35700] Loss: 0.063375 Acc: 0.9844
	Test set: Average loss: 0.0729, Accuracy: 6179/6300 (98%)
Removing old model C:\Users\fdal\Desktop\bigs\TOBIGS\best-4.pth
Saving model to C:\Users\fdal\Desktop\bigs\TOBIGS\best-5.pth
Train Epoch: 6 [1280/35700] Loss: 0.071985 Acc: 0.9766
Train Epoch: 6 [2560/35700] Loss: 0.094118 Acc: 0.9688
Train Epoch: 6 [3840/35700] Loss: 0.086459 Acc: 0.9688
Train Epoch: 6 [5120/35700] Loss: 0.114306 Acc: 0.9688
Train Epoch: 6 [6400/35700] Loss: 0.128841 Acc: 0.9766
Train Epoch: 6 [7680/35700] Loss: 0.065532 Acc: 0.9844
Train Epoch: 6 [8960/35700] Loss: 0.101370 Acc: 0.9609
Train Epoch: 6 [10240/35700] Loss: 0.073002 Acc: 0.9688
Train Epoch: 6 [11520/35700] Loss: 0.083907 Acc: 0.9766
Train Epoch: 6 [12800/35700] Loss: 0.051260 Acc: 0.9844
Train Epoch: 6 [14080/35700] Loss: 0.106676 Acc: 0.9766
Train Epoch: 6 [15360/35700] Loss: 0.05

In [26]:
for epoch in range(20):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            indx_target = target.clone()
            data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)

            optimizer.zero_grad()
            output = model(data)
#             output = nn.Softmax(output)
#             loss = F.mse_loss(output, target)
            loss=F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 10 ==0 and batch_idx > 0:
                pred = output.data.max(1)[1]  # get the index of the max log-probability
                correct = pred.cpu().eq(indx_target).sum()
                acc = correct * 1.0 / len(data)
                print('Train Epoch: {} [{}/{}] Loss: {:.6f} Acc: {:.4f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    loss, acc))

        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            indx_target = target.clone()
            data, target = data.cuda(), target.cuda()
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            test_loss += F.cross_entropy(output, target)
            pred = output.data.max(1)[1]
            correct += pred.cpu().eq(indx_target).sum()

        test_loss = test_loss / len(test_loader) 
        acc = 100. * correct / float(len(test_loader.dataset))
        print('\tTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            test_loss, correct, len(test_loader.dataset), acc))
        
        if acc > best_acc:
            new_file = os.path.join('./', 'best3-{}.pth'.format(epoch))
            model_snapshot(model, new_file, old_file=old_file)
            best_acc = acc
            old_file = new_file

Train Epoch: 0 [1280/35700] Loss: 0.013202 Acc: 1.0000
Train Epoch: 0 [2560/35700] Loss: 0.035058 Acc: 0.9922
Train Epoch: 0 [3840/35700] Loss: 0.016134 Acc: 0.9922
Train Epoch: 0 [5120/35700] Loss: 0.019590 Acc: 0.9922
Train Epoch: 0 [6400/35700] Loss: 0.031274 Acc: 0.9922
Train Epoch: 0 [7680/35700] Loss: 0.023607 Acc: 0.9922
Train Epoch: 0 [8960/35700] Loss: 0.044469 Acc: 0.9844
Train Epoch: 0 [10240/35700] Loss: 0.017271 Acc: 1.0000
Train Epoch: 0 [11520/35700] Loss: 0.005256 Acc: 1.0000
Train Epoch: 0 [12800/35700] Loss: 0.010144 Acc: 1.0000
Train Epoch: 0 [14080/35700] Loss: 0.036877 Acc: 0.9844
Train Epoch: 0 [15360/35700] Loss: 0.034577 Acc: 0.9766
Train Epoch: 0 [16640/35700] Loss: 0.024325 Acc: 0.9922
Train Epoch: 0 [17920/35700] Loss: 0.004251 Acc: 1.0000
Train Epoch: 0 [19200/35700] Loss: 0.016439 Acc: 1.0000
Train Epoch: 0 [20480/35700] Loss: 0.013770 Acc: 0.9922
Train Epoch: 0 [21760/35700] Loss: 0.052215 Acc: 0.9922
Train Epoch: 0 [23040/35700] Loss: 0.022394 Acc: 0.9922

C:\Users\fdal\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


	Test set: Average loss: 0.0464, Accuracy: 6219/6300 (99%)
Train Epoch: 1 [1280/35700] Loss: 0.009305 Acc: 1.0000
Train Epoch: 1 [2560/35700] Loss: 0.006396 Acc: 1.0000
Train Epoch: 1 [3840/35700] Loss: 0.015711 Acc: 1.0000
Train Epoch: 1 [5120/35700] Loss: 0.013309 Acc: 0.9922
Train Epoch: 1 [6400/35700] Loss: 0.029237 Acc: 0.9844
Train Epoch: 1 [7680/35700] Loss: 0.002397 Acc: 1.0000
Train Epoch: 1 [8960/35700] Loss: 0.019866 Acc: 0.9922
Train Epoch: 1 [10240/35700] Loss: 0.009061 Acc: 1.0000
Train Epoch: 1 [11520/35700] Loss: 0.019840 Acc: 0.9922
Train Epoch: 1 [12800/35700] Loss: 0.010205 Acc: 1.0000
Train Epoch: 1 [14080/35700] Loss: 0.011645 Acc: 1.0000
Train Epoch: 1 [15360/35700] Loss: 0.023589 Acc: 0.9844
Train Epoch: 1 [16640/35700] Loss: 0.024232 Acc: 0.9922
Train Epoch: 1 [17920/35700] Loss: 0.016581 Acc: 1.0000
Train Epoch: 1 [19200/35700] Loss: 0.006078 Acc: 1.0000
Train Epoch: 1 [20480/35700] Loss: 0.044236 Acc: 0.9922
Train Epoch: 1 [21760/35700] Loss: 0.008707 Acc: 1.0

Train Epoch: 6 [6400/35700] Loss: 0.008076 Acc: 1.0000
Train Epoch: 6 [7680/35700] Loss: 0.007146 Acc: 1.0000
Train Epoch: 6 [8960/35700] Loss: 0.024096 Acc: 0.9922
Train Epoch: 6 [10240/35700] Loss: 0.010962 Acc: 1.0000
Train Epoch: 6 [11520/35700] Loss: 0.007814 Acc: 1.0000
Train Epoch: 6 [12800/35700] Loss: 0.022170 Acc: 0.9922
Train Epoch: 6 [14080/35700] Loss: 0.023966 Acc: 0.9922
Train Epoch: 6 [15360/35700] Loss: 0.004396 Acc: 1.0000
Train Epoch: 6 [16640/35700] Loss: 0.007671 Acc: 1.0000
Train Epoch: 6 [17920/35700] Loss: 0.019449 Acc: 0.9922
Train Epoch: 6 [19200/35700] Loss: 0.027812 Acc: 0.9922
Train Epoch: 6 [20480/35700] Loss: 0.022467 Acc: 0.9922
Train Epoch: 6 [21760/35700] Loss: 0.020296 Acc: 0.9922
Train Epoch: 6 [23040/35700] Loss: 0.008572 Acc: 1.0000
Train Epoch: 6 [24320/35700] Loss: 0.015058 Acc: 1.0000
Train Epoch: 6 [25600/35700] Loss: 0.002481 Acc: 1.0000
Train Epoch: 6 [26880/35700] Loss: 0.008563 Acc: 1.0000
Train Epoch: 6 [28160/35700] Loss: 0.011974 Acc: 1.

Train Epoch: 11 [8960/35700] Loss: 0.012794 Acc: 1.0000
Train Epoch: 11 [10240/35700] Loss: 0.008896 Acc: 1.0000
Train Epoch: 11 [11520/35700] Loss: 0.009799 Acc: 1.0000
Train Epoch: 11 [12800/35700] Loss: 0.021937 Acc: 0.9922
Train Epoch: 11 [14080/35700] Loss: 0.056562 Acc: 0.9922
Train Epoch: 11 [15360/35700] Loss: 0.006776 Acc: 1.0000
Train Epoch: 11 [16640/35700] Loss: 0.004502 Acc: 1.0000
Train Epoch: 11 [17920/35700] Loss: 0.002006 Acc: 1.0000
Train Epoch: 11 [19200/35700] Loss: 0.007799 Acc: 1.0000
Train Epoch: 11 [20480/35700] Loss: 0.004900 Acc: 1.0000
Train Epoch: 11 [21760/35700] Loss: 0.017317 Acc: 0.9922
Train Epoch: 11 [23040/35700] Loss: 0.006592 Acc: 1.0000
Train Epoch: 11 [24320/35700] Loss: 0.003974 Acc: 1.0000
Train Epoch: 11 [25600/35700] Loss: 0.003421 Acc: 1.0000
Train Epoch: 11 [26880/35700] Loss: 0.007707 Acc: 1.0000
Train Epoch: 11 [28160/35700] Loss: 0.002051 Acc: 1.0000
Train Epoch: 11 [29440/35700] Loss: 0.005310 Acc: 1.0000
Train Epoch: 11 [30720/35700] Lo

Train Epoch: 16 [11520/35700] Loss: 0.005164 Acc: 1.0000
Train Epoch: 16 [12800/35700] Loss: 0.003157 Acc: 1.0000
Train Epoch: 16 [14080/35700] Loss: 0.005262 Acc: 1.0000
Train Epoch: 16 [15360/35700] Loss: 0.008923 Acc: 1.0000
Train Epoch: 16 [16640/35700] Loss: 0.004938 Acc: 1.0000
Train Epoch: 16 [17920/35700] Loss: 0.004617 Acc: 1.0000
Train Epoch: 16 [19200/35700] Loss: 0.002539 Acc: 1.0000
Train Epoch: 16 [20480/35700] Loss: 0.006119 Acc: 1.0000
Train Epoch: 16 [21760/35700] Loss: 0.015102 Acc: 0.9922
Train Epoch: 16 [23040/35700] Loss: 0.002279 Acc: 1.0000
Train Epoch: 16 [24320/35700] Loss: 0.002649 Acc: 1.0000
Train Epoch: 16 [25600/35700] Loss: 0.004936 Acc: 1.0000
Train Epoch: 16 [26880/35700] Loss: 0.014666 Acc: 0.9922
Train Epoch: 16 [28160/35700] Loss: 0.003947 Acc: 1.0000
Train Epoch: 16 [29440/35700] Loss: 0.029122 Acc: 0.9922
Train Epoch: 16 [30720/35700] Loss: 0.005567 Acc: 1.0000
Train Epoch: 16 [32000/35700] Loss: 0.021152 Acc: 0.9922
Train Epoch: 16 [33280/35700] L

In [27]:
test

,Unnamed: 0,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,57808,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4960,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,35755,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15543,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,48968,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,53718,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17996,78,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17997,33373,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17998,36134,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
X_test = torch.tensor(test.iloc[:,1:].values / 255,dtype=torch.float)

In [51]:
X_test

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [52]:
best_model = TOBIGS(input_dims=784, n_hiddens=[256, 256], n_class=10)
best_model.load_state_dict(torch.load('./best3-14.pth'))
best_model.cuda()
best_model.eval()
X_test = Variable(X_test.cuda())
pred = best_model(X_test)

Sequential(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0.2, inplace=False)
  (out): Linear(in_features=256, out_features=10, bias=True)
)


In [57]:
sub = pd.read_csv("sample_submission.csv")

In [61]:
sub["Category"] = pred.argmax(axis=1).cpu().numpy()

In [65]:
sub.to_csv("torch_pred.csv")